## 1. Web Crawling for Stock Related News

## 2. Scrolling Down Page to Get More New

## 3. Text Mining

## 4.  Acquiring Stock Data

In [7]:
import bs4 as bs
import requests

def get_sp500_codes():
    sp500 = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    soup = bs.BeautifulSoup(sp500.text, 'lxml')
    table = soup.find('table', {'class': 'wikitable sortable'})
    codes = []
    for row in table.findAll('tr')[1:]:
        code = row.findAll('td')[0].text
        codes.append(code)

    return codes

In [8]:
stock_code = get_sp500_codes()

In [ ]:
def get_sentiment(stock_code):
    sentiment_com_tr = []
    sentiment_pos_tr = []
    sentiment_neg_tr = []
    sentiment_com_te = []
    sentiment_pos_te = []
    sentiment_neg_te = []
    error_code = []
    for i in stock_code:
        try:
            all_links = get_article_link(i)
            artical_content = get_article_content(all_links)
            texts = artical_format(artical_content)
            a = vader_comparison(texts)
            a['Name'] = pd.to_datetime(a['Name'])
            a = a.sort_values(['Name'],ascending = 0)
            a_train = a[-round(len(a)*0.7):]
            a_test = a[:round(len(a)*0.3)]
            sentiment_com_tr.append(a_train.mean()['compound'])
            sentiment_pos_tr.append(a_train.mean()['pos'])
            sentiment_neg_tr.append(a_train.mean()['neg'])
            sentiment_com_te.append(a_test.mean()['compound'])
            sentiment_pos_te.append(a_test.mean()['pos'])
            sentiment_neg_te.append(a_test.mean()['neg'])
        except:
            error_code.append(i)
    return [sentiment_com_tr,sentiment_pos_tr,sentiment_neg_tr,sentiment_com_te,sentiment_pos_te,sentiment_neg_te,error_code]

In [ ]:
import pickle
for i in range(5):
    sentiment = get_sentiment(stock_code[i*51:(i+1)*51])
    name = "sentiment"+str(i)+".pc"
    file = open(name,'wb')
    pickle.dump(sentiment,file)
    file.close()
    print('sentiment',i,'done')

In [ ]:
import numpy as np
import statsmodels.api as sm
X = np.column_stack((sentiment[0], sentiment[1],sentiment[2]))
X = sm.add_constant(X)
y = stock_ret
model = sm.OLS(y,X)
results = model.fit()
print(results.params)


In [ ]:
def prediction(stock_code):
    if isinstance(stock_code, str):
        stock_code = [stock_code] 
    result = []
    for i in range(len(stock_code)):
        predict_return = np.dot([1, sentiment[3][i],sentiment[4][i],sentiment[5][i]],results.params)
        result.append(predict_return)
    return result

## 5. Data Visualization